# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 57 new papers today
          24 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/23 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.06226


extracting tarball to tmp_2209.06226...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.06227


extracting tarball to tmp_2209.06227...

...

extracting tarball to tmp_2209.06741...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.06766


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.06741/Lai_NGC7469.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.06766...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06226-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06226) | **Gravitational wave fossils in nonlinear regime: halo tidal bias and  intrinsic alignments from gravitational wave separate universe simulations**  |
|| Kazuyuki Akitsu, <mark>Yin Li</mark>, Teppei Okumura |
|*Appeared on*| *2022-09-15*|
|*Comments*| *27 pages, 17 figures, comments are welcome!*|
|**Abstract**| We investigate impacts of long-wavelength gravitational waves (GWs) on nonlinear structure formation by utilizing the tidal separate universe simulations. Based on the equivalence of a long-wavelength GW to a uniform tidal field in a local frame, we provide a way to incorporate a long-wavelength GW into the tidal separate universe simulation as an effective anisotropic expansion. This methodology enables us to study effects of GWs on large-scale structure efficiently. We measure the anisotropic imprint in the local power spectrum from the tidal separate universe simulations with GWs, which corresponds to the scalar-scalar-tensor bispectrum in squeezed limit or the so-called power spectrum response to GWs. We also detect the halo tidal bias induced by GWs from the response of the halo-matter cross-power spectrum to GWs, as well as the linear shape bias (or the linear alignment coefficient) induced by GWs from the one-point function of the halo ellipticity. In contrast to the case of the tidal field induced by scalar perturbations, we discover that the wavenumber dependence of the temporal evolution of GWs naturally causes these biases to be scale-dependent. We also find that this scale dependence is well approximated by the second-order density induced by the coupling between scalar and tensor perturbation. This highlights that the structure formation, especially the process to determine the halo shape, is nonlocal in time. Our findings lay the foundation for predicting the impact of GWs on large-scale structure. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06227-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06227) | **Polarized Blazar X-rays imply particle acceleration in shocks**  |
|| <mark>Ioannis Liodakis</mark>, et al. -- incl., <mark>Henric Krawczynski</mark>, <mark>Elina Lindfors</mark>, <mark>Herman L. Marshall</mark>, <mark>Joseph R. Masiero</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| *27 pages, 4 figures, 4 tables, author's version of the paper accepted for publication in Nature*|
|**Abstract**| Blazars are active galactic nuclei that launch collimated, powerful jets of magnetized relativistic plasma. Their primary jet, whose emission typically spans from low-frequency radio to very high-energy ($\gtrsim0.1$ TeV) $\gamma$-rays (Blandford et al., 2019), is aligned towards our line of sight. Multiwavelength polarization is a crucial probe of the magnetic field structure and emission processes in such jets. Until now, sensitive polarization observations have been limited to the radio, infrared, and optical range, thereby leaving a gap in our knowledge of the physical conditions experienced by the most energetic particles. Here, we report the first-ever detection of X-ray polarization from the jet in an accreting supermassive black hole system, the blazar Markarian 501 (Mrk 501). The recently launched Imaging X-ray Polarimetry Explorer ($IXPE$, Weisskopf et al., 2022) measures a linear polarization degree ($\Pi$) over the 2-8 keV X-ray energy range of 10$\pm$2% with an electric vector position angle of 134$^\circ\pm$5$^\circ$, parallel to the radio jet. The X-ray $\Pi$ is more than a factor of 2 higher than the optical $\Pi$. We conclude that an energy-stratified relativistic electron population, i.e., an acceleration scenario where the higher energy particles emit from more magnetically ordered regions closer to the acceleration site, is the most likely explanation of the higher degree of polarization at X-ray energies. A second $IXPE$ observation conducted 16 days later yielded similar results, strengthening our conclusions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06229-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06229) | **A Deep View into the Nucleus of the Sagittarius Dwarf Spheroidal Galaxy  with MUSE. III. Discrete multi-component population-dynamical models based on  the Jeans equations**  |
|| Nikolay Kacharov, et al. -- incl., <mark>Mayte Alfaro-Cuello</mark>, <mark>Nadine Neumayer</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| We present comprehensive multi-component dynamical models of M54 (NGC6715), the nuclear star cluster of the Sagittarius dwarf galaxy (Sgr), which is undergoing a tidal disruption in the Milky Way halo. Previous papers in the series used a large MUSE mosaic data set to identify multiple stellar populations in the system and study their kinematic differences. Here we use Jeans-based dynamical models that fit the population properties (mean age and metallicity), spatial distributions, and kinematics simultaneously. They provide a solid physical explanation to our previous findings. The population-dynamical models deliver a comprehensive view of the whole system, and allow us to disentangle the different stellar populations. We explore their dynamical interplay and confirm our previous findings about the build-up of Sgr's nuclear cluster via contributions from globular cluster stars, Sgr inner field stars, and in-situ star formation. We explore various parameterisations of the gravitational potential and show the importance of a radially varying mass-to-light ratio for the proper treatment of the mass profile. We find a total dynamical mass within M54's tidal radius ($\sim75$ pc) of $1.60\pm0.07\times10^6$ Msun in excellent agreement with $N$-body simulations. The metal-poor globular cluster stars contribute about $65\%$ of the total mass or $1.04\pm0.05\times10^6$ Msun. The metal-rich stars can be further divided into young and intermediate age populations that contribute $0.32\pm0.02\times10^6$ Msun ($20\%$) and $0.24\pm0.02\times10^6$ Msun ($15\%$), respectively. Our population-dynamical models successfully distinguish the different stellar populations in Sgr's nucleus because of their different spatial distributions, ages, metallicities, and kinematic features. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06513-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06513) | **Low-temperature optical constants of amorphous silicate dust analogues**  |
|| K. Demyk, et al. -- incl., <mark>P. Hubert</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| **|
|**Abstract**| Cosmic dust models are key ingredients in advancing our understanding of astronomical environments as diverse as interstellar clouds in galaxies, circumstellar envelopes around evolved and young stars, and protoplanetary disks. Such models consist of several dust populations, each with different compositions and size distributions. They may also consider different grain shapes, although most models assume spherical grains. All include a component of silicate dust. The absorption and emission properties of these dust components are calculated from the optical constants of each dust material which have various experimental, phenomenological, and theoretical origins depending on the model. We aim to provide the community with new sets of optical constants for amorphous silicate dust analogues at low temperatures. The analogues consist of four Mg-rich silicate samples of stoichiometry ranging from enstatite to olivine, and of eight samples of Mg and Fe rich silicates with a pyroxene stoichiometry and differing magnesium and iron content. We calculated the optical constants from transmission measurements using the Kramers-Kronig relations, assuming that the grains are small compared to the wavelength and prolate in shape with axis ratios of 1.5 and 2 for the Mg and Fe rich samples, respectively. New optical constants for silicate dust analogues were calculated over the wavelength range from 5 to 800-1000 microns, depending on the sample, and at temperatures of 10, 30, 100, 200, and 300 K.We determined the uncertainties on the derived optical constants based on the assumptions used to calculate them. To facilitate the use of these data in cosmic dust models, we provide optical constants extrapolated outside the measured spectral range into the UV-NIR and mm-cm wavelength ranges, as well as formulae that can be used to interpolate them at any temperature in the range 10 - 300 K. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06714-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06714) | **Revisiting the Evidence for an Intermediate-mass Black Hole in the  Center of NGC 6624 with Simulations**  |
|| <mark>Li-Chun Wang</mark>, <mark>Yi Xie</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| *10 pages, 3 figures; accepted for publication in Research in Astronomy and Astrophysics*|
|**Abstract**| The acceleration of LMXB 4U 1820-30 that derived from its orbital-period derivative $\dot P_{\rm b}$ was supposed to be the evidence for an Intermediate-mass Black Hole (IMBH) in the Galactic globular cluster (GC) NGC 6624. However, we find that the anomalous $\dot P_{\rm b}$ is mainly due to the gravitational wave emission, rather than the acceleration in cluster potential. Using the standard structure models of GCs, we simulate acceleration distributions for pulsars in the central region of the cluster. By fitting the acceleration of J1823-3021A with the simulated distribution profiles (maximum values), it is suggested that an IMBH with mass $M\gtrsim 950^{+550}_{-350}~M_{\odot}$ may reside in the cluster center. We further show that the second period derivative $\ddot P$ of J1823-3021A is probably due to the gravitational perturbation of a nearby star. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xa8 in position 35860: invalid start byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06288-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06288) | **Harmonic Electron Cyclotron Maser Emission along the Coronal Loop**  |
|| Mehdi Yousefzadeh, Yao Chen, Hao Ning, <mark>Mahboub Hosseinpour</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| **|
|**Abstract**| Efficient radiation at second and/or higher harmonics of Wce has been suggested to circumvent the escaping difficulty of the electron cyclotron maser emission mechanism when it is applied to solar radio bursts, such as spikes. In our earlier study, we developed a three-step numerical scheme to connect the dynamics of energetic electrons within a large-scale coronal loop structure with the microscale kinetic instability energized by the obtained nonthermal velocity distribution and found that direct and efficient harmonic X-mode (X2 for short) emission can be achieved due to the strip-like features of the distribution. That study only considered the radiation from the loop top at a specific time. Here we present the emission properties along the loop at different locations and timings. We found that, in accordance with our earlier results, few to several strip-like features can appear in all cases, and the first two strips play the major role in exciting X2 and Z (i.e., the slow extraordinary mode) that propagate quasi-perpendicularly. For the four sections along the loop, significant excitation of X2 is observed from the upper two sections, and the strongest emission is from the top section. In addition, significant excitation of Z is observed for all loop sections, while there is no significant emission of the fundamental X mode. The study provides new insight into coherent maser emission along the coronal loop structure during solar flares. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06659-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06659) | **Effects of impact and target parameters on the results of a kinetic  impactor: predictions for the Double Asteroid Redirection Test (DART) mission**  |
|| Angela M. Stickle, et al. -- incl., <mark>Mallory E. DeCoster</mark>, <mark>Yun Zhang</mark>, <mark>M.Isabel Herreros</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| *Accepted to PSJ Didymos-DART Focus Issue*|
|**Abstract**| The Double Asteroid Redirection Test (DART) spacecraft will impact into the asteroid Dimorphos on September 26, 2022 as a test of the kinetic impactor technique for planetary defense. The efficiency of the deflection following a kinetic impactor can be represented using the momentum enhancement factor, Beta, which is dependent on factors such as impact geometry and the specific target material properties. Currently, very little is known about Dimorphos and its material properties that introduces uncertainty in the results of the deflection efficiency observables, including crater formation, ejecta distribution, and Beta. The DART Impact Modeling Working Group (IWG) is responsible for using impact simulations to better understand the results of the DART impact. Pre-impact simulation studies also provide considerable insight into how different properties and impact scenarios affect momentum enhancement following a kinetic impact. This insight provides a basis for predicting the effects of the DART impact and the first understanding of how to interpret results following the encounter. Following the DART impact, the knowledge gained from these studies will inform the initial simulations that will recreate the impact conditions, including providing estimates for potential material properties of Dimorphos and Beta resulting from DARTs impact. This paper summarizes, at a high level, what has been learned from the IWG simulations and experiments in preparation for the DART impact. While unknown, estimates for reasonable potential material properties of Dimorphos provide predictions for Beta of 1-5, depending on end-member cases in the strength regime. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.06247-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.06247) | **CLEAR: High-Ionization [Ne V] $λ$3426 Å Emission-line  Galaxies at $1.4 <z< 2.3$**  |
|| Nikko J. Cleri, et al. -- incl., <mark>Jasleen Matharu</mark>, <mark>Ivelina Momcheva</mark> |
|*Appeared on*| *2022-09-15*|
|*Comments*| *17 pages + 2 (appendix), 8 figures + 2(appendix)*|
|**Abstract**| We analyze a sample of 25 [Ne V] (${\lambda}$3426 $\r{A}$) emission-line galaxies at 1.4 < z < 2.3 using Hubble Space Telescope/Wide Field Camera 3 G102 and G141 grism observations from the CANDELS Lyman-${\alpha}$ Emission at Reionization (CLEAR) survey. [Ne V] emission probes extremely energetic photoionization (creation potential of 97.11 eV), and is often attributed to energetic radiation from active galactic nuclei (AGN), radiative supernova shocks, or an otherwise very hard ionizing spectrum from the stellar continuum. In this work, we use [Ne V] in conjunction with other rest-frame UV/optical emission lines ([O II] ${\lambda\lambda}$3726,3729 $\r{A}$, [Ne III] ${\lambda}$3869 $\r{A}$, [O III] ${\lambda\lambda}$4959,5007 $\r{A}$, H${\alpha}$+[N II] ${\lambda\lambda}$6548,6583 $\r{A}$, [S II] ${\lambda\lambda}$6716,6731 $\r{A}$), deep (2-7 Ms) X-ray observations (from Chandra), and mid-infrared imaging (from Spitzer) to study the origin of this emission and to place constraints on the nature of the ionizing engine. The majority of the [Ne V]-detected galaxies have properties consistent with ionization from AGN. However, for our [Ne V]-selected sample, the X-ray luminosities are consistent with local (z < 0.1) X-ray-selected Seyferts, but the [Ne V] luminosities are more consistent with those from z ~ 1 X-ray-selected QSOs. The excess [Ne V] emission requires either reduced hard X-rays, or a ~0.1 keV excess. We discuss possible origins of the apparent [Ne V] excess, which could be related to the "soft (X-ray) excess" observed in some QSOs and Seyferts, and/or be a consequence of a complex/anisotropic geometry for the narrow line region, combined with absorption from a warm, relativistic wind ejected from the accretion disk. We also consider implications for future studies of extreme high-ionization systems in the epoch of reionization (z > 6) with the James Webb Space Telescope. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

32  publications in the last 7 days.
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers